# Elementos Frecuentes, reglas de asociación, algoritmo apriori 

Vamos a calcular reglas de asociación usando el paquete **efficient-apriori**, cuya documentación se encuentra en https://pypi.org/project/efficient-apriori/. Con este paquete, vamos a procesar ejemplos y discutir los conceptos vistos en clases

## Instalación, usos básicos

In [ ]:
### Usar pip para cargar el paquete en nuestro ambiente actual - omitir si el paquete ya está instalado!
import sys
!{sys.executable} -m pip install efficient-apriori


In [1]:
### Un primer uso (extraído de la documentación oficial): 

from efficient_apriori import apriori 

### Cargamos las transacciones como una lista, con elementos. En este caso son tres compras.  

transactions = [('eggs', 'bacon', 'soup'),('eggs', 'bacon', 'apple'),('soup', 'bacon', 'banana')]

### Y corremos nuestro modelo. Definimos el soporte como 0.6, y la confianza en 1. 

itemsets, rules = apriori(transactions, min_support=0.6, min_confidence=1) 




In [2]:
### Imprimimos las reglas. 

### En este caso, como la confianza es 1, tomamos solo las reglas que siempre se cumplen en todas las canastas 
### donde aparece el lado izquierdo.

### El soporte de 0.66 requiere que al menos 2 de 3 de las canastas soporten esta regla, es decir, que tengan 
### los elementos de ambos lados de la regla.
print(rules) 


[{eggs} -> {bacon}, {soup} -> {bacon}]


In [3]:
### Podemos imprimir también el conteo de todos los conjuntos de elementos que tienen soporte sobre el 
### minimo que definimos, en este caso 0.6

print(itemsets) 

{1: {('eggs',): 2, ('bacon',): 3, ('soup',): 2}, 2: {('bacon', 'eggs'): 2, ('bacon', 'soup'): 2}}


In [12]:
### Y con esto calculamos el interes de ciertas reglas. Por ejemplo, el interes de {eggs} -> {bacon} es 
(itemsets[2][('bacon', 'eggs')]/itemsets[1][('eggs',)]) - (itemsets[1][('bacon',)]/3)

# Lo que nos dice que en realidad esta regla no es interesante. Si lo pensamos, es obvia, siempre hay bacon! 

0.0

In [47]:
### Veamos que pasa al cambiar un poco las canastas, bajamos tb el soporte y la confianza

tr = [('eggs', 'bacon', 'soup'),('eggs', 'pear', 'apple'),('soup', 'bacon', 'banana'),('apple', 'banana','soup')]

itemsets, rules = apriori(tr, min_support=0.5, min_confidence=0.6) 
print(rules) 



[{soup} -> {bacon}, {bacon} -> {soup}, {soup} -> {banana}, {banana} -> {soup}]


In [48]:
### ¿Cual regla es mas interesante, {soup} -> {bacon} o {bacon} -> {soup}? ¿Puedes ver por qué?


## Desde csv a apriori

Vamos a cargar un conjunto de datos almacenados en un archivo *canastas.csv*. Los cargaremos a una lista llamada datos, que tiene una tupla con los elementos por lista. 

In [2]:
import csv

with open('canastas.csv', 'r', newline='', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    datos = [tuple(row) for row in reader]
    


In [3]:
len(datos)

1001

In [17]:
print(datos[0])

('salsa barbacoa', 'uvas', 'salsa alioli', 'sal', 'judías verdes', 'moras', 'manzanas', 'guisantes', 'orégano', 'maíz', 'peras', 'arroz', 'aguacates', 'bacalao', 'espárragos')


Ahora usamos estos datos en el código que teníamos antes. 

In [18]:
transactions = datos
itemsets, rules = apriori(transactions, min_support=0.66, min_confidence=1) 


In [19]:
print(rules)

[]


¿Qué pasó? Las listas involucran muchos elementos, y ya es muy poco probable que una regla tenga confianza de 1. 

Recordemos: El soporte es el porcentaje de las canastas donde si aplica esta regla. Un umbral de soporte más alto filtra más, y por lo tanto se generan menos reglas. 

La confianza es la proporcion de canastas que tienen el lado izquierdo de la regla, y el lado derecho, contra las canastas que solo tienen el lado izquierdo. Un umbral de confianza más alto filtra más, y por lo tanto se generan menos reglas. 

En el retail es usual ver un soporte de 0.01 a 0.05 (osea, buscar reglas que se apliquen en un 1%-5% de las canastas), y una confianza idealmente mayor a 0.5. Pero estos valores siempre van a depender de la naturaleza de los datos. 

Veamos qué pasa si ahora bajamos el soporte a 0.01 y la confianza a 0.3

In [22]:
transactions = datos
itemsets, rules = apriori(transactions, min_support=0.01, min_confidence=0.3) 

print(rules)

[{tomates} -> {ajo}]


Parece ser que tomates y ajo van bien juntos! 
Esta regla nos dice que
- (por el soporte mínimo): al menos el 1% de las compras incluyen tomates y ajos 
- (por la confianza mínima): al menos un 30% de las compras que incluyen tomates incluyen también ajos. 

Puedes jugar con los umbrales de soporte y confianza, y descubrir nuevas reglas de asociación? Lógiamente, este descubrimiento tiene que involucrar un menor umbral de soporte, o una menor confianza, o ambas. 

## Actividad

Implementa, por sobre la librería para apriori, alguno de los otros dos algoritmos que vimos en clases y que construyen sobre la idea de trabajar con samples o dividir todas las canastas en muchos pedazos.  